In [168]:
#imports
from bs4 import BeautifulSoup
import urllib
from urllib.request import urlopen

import mutagen
from mutagen.easyid3 import EasyID3

import os
import re

dir_path="/home/naisargee/Music/W300i/practice/"
change_count=0
change_files=[]

In [169]:
regex = re.compile('[^a-zA-Z]')
def search_term_from_file(file):
    print("search Term :",regex.sub(' ', file[:-3]))
    return regex.sub(' ', file[:-3]).strip()

    
def google_output_string(file):
    song=search_term_from_file(file)
    url = 'https://www.google.com/search?q='+song.replace(" ","+")
    headers = {}
    headers['User-Agent'] = "Mozilla/5.0 (X11; Linux i686) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1312.27 Safari/537.17"
    req = urllib.request.Request(url, headers = headers)
    resp = urllib.request.urlopen(req)
    respData = resp.read()
    soup = BeautifulSoup(str(respData))
    k=soup.find("div", {"class": "_G1d _wle _xle"})
    value=k.get_text()
    print("from google :",value)
    return value

def find_between_strings( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""


def extract_dic_para(file):
    string=google_output_string(file)
    dic={}
    if 'Artists:' in string:
        dic["Artists"]=find_between_strings(string,"Artists:","Movie:").strip()
    elif 'Artist' in string:
        dic["Artists"]=find_between_strings(string,"Artist:","Movie:").strip()
    dic["Movie"]=find_between_strings(string,"Movie:","Released:").strip()
    temp=string.index("Released: ")+len("Released: ")
    dic["Released"]=string[temp:temp+4]
    print("parameter extracted :",dic)
    return dic


        

In [170]:
def list_of_files_in_dir(path=dir_path):
    l=[]
    for file in os.listdir(path):
        if file.endswith(".mp3"):
            l.append(file)
    print("total_number_of_files :",len(l))
    return l

def edit_details_of_file(file):
    global change_count
    global change_file
    audio = EasyID3(dir_path+file)
    print(dir_path+file)
    print("type",type(audio))
    dic=extract_dic_para(file)
    print(dic)
    print(audio["artist"],audio["album"],audio["date"])
    audio["artist"] = unicode(dic["Artists"], "ascii")
    audio["album"] = unicode(dic["Movie"], "ascii")
    audio["date"] = unicode(dic["Released"], "ascii")
    print(audio["artist"],audio["album"],audio["date"])
    audio.save()
    change_count=change_count+1
    change_files.append(file)
    
def apply_on_directory():
    files=list_of_files_in_dir()
    for file in files:
        try:
            print(file)
            edit_details_of_file(file)
        except Exception as e:
            print(e)
            print(type(e))
            continue

In [172]:
change_count=0
apply_on_directory()

total_number_of_files : 2
Aao Milo Chale.mp3
/home/naisargee/Music/W300i/practice/Aao Milo Chale.mp3
type <class 'mutagen.easyid3.EasyID3'>
search Term : Aao Milo Chale 
from google : Artists: Shaan, Sultan KhanMovie: Jab We MetReleased: 2007
parameter extracted : {'Movie': 'Jab We Met', 'Artists': 'Shaan, Sultan Khan', 'Released': '2007'}
{'Movie': 'Jab We Met', 'Artists': 'Shaan, Sultan Khan', 'Released': '2007'}
['www.Songs.PK'] ['Jab We Met'] ['2007']
name 'unicode' is not defined
<class 'NameError'>
AAOGE JAB TUM.mp3
/home/naisargee/Music/W300i/practice/AAOGE JAB TUM.mp3
type <class 'mutagen.easyid3.EasyID3'>
search Term : AAOGE JAB TUM 
from google : Artist: Rashid KhanMovie: Jab We MetReleased: 2007
parameter extracted : {'Movie': 'Jab We Met', 'Artists': 'Rashid Khan', 'Released': '2007'}
{'Movie': 'Jab We Met', 'Artists': 'Rashid Khan', 'Released': '2007'}
'TPE1'
<class 'KeyError'>


In [ ]:
print(change_count)
print(change_files)